# VALEURS DE DOUBLONS D'AIS

In [31]:
import pandas as pd
import numpy as np

# Read data

In [32]:
spire_positions_file_path = "../../spire_positions_subset_02022024.csv"

In [33]:
spire_positions_df = pd.read_csv(
    spire_positions_file_path,
    parse_dates=[
        "timestamp",
        "last_position_time",
        "voyage_eta"
    ],
    dtype={
        "IMO": str,
        "mmsi": str,
    }
)

# Analysis

In [34]:
#Grouper les données par noms de bateaux
ship=spire_positions_df.groupby('ship_name')
all_ship_data = pd.concat([group for _, group in ship])

#Créer un Dataframe contenant seules les lignes contenant des renvoie d'AIS non actualisées groupées par ship_name
def find_duplicates(group):
    return group[group.duplicated(subset='last_position_time', keep=False)]

duplicates = ship.apply(find_duplicates)

/tmp/ipykernel_12501/652334267.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duplicates = ship.apply(find_duplicates)


In [35]:
import pandas as pd
#Créer un Dataframe contenant seules les valeurs d'AIS correctement actualisées groupées par ship_name
duplicates1=duplicates.copy()
duplicates1.reset_index(drop=True, inplace=True)
non_duplicates = pd.merge(all_ship_data, duplicates1, how='left', indicator=True)
non_duplicates = non_duplicates[non_duplicates['_merge'] == 'left_only']
non_duplicates.drop(columns=['_merge'], inplace=True)

#la colonne occurence contient le nombre d'occurence observée pour une valeur repetée d'ais, et doublon renseigne si la ligne en question est un doublon ou non.
#on renseigne ici dans la colonne doublon 'actualisée' car ce le dataframe non_duplicate contient uniquement des valeurs d'ais non répétées et donc logiquement 0 dans occurences 
non_duplicates['occurences']= 0
non_duplicates['doublons']= 'actualisée'

In [36]:
############Création des colonnes occurences et doublons dans le dataframe de valeurs d'ais mal actualisées################################

duplicates['occurences'] = duplicates.groupby('last_position_time')['last_position_time'].transform('count')
duplicates['doublons'] = np.where(duplicates.duplicated(subset='last_position_time', keep=False), 'actualisée', 'réplicat')

# Marquer la première occurrence de chaque données AIS comme 'actualisée'
first_occurrence_mask = duplicates.duplicated(subset='last_position_time', keep='first')
duplicates.loc[first_occurrence_mask, 'doublons'] = 'réplicat'

result = duplicates

In [37]:
#result.xs('UK153 LUB"B!VYOR"', level=0)

In [38]:
#réorganisation du Dataframe result afin de coller à la structure de non_duplicates 
result1 = result.drop('ship_name', axis=1)
result1.reset_index(level='ship_name', inplace=True)
ship_name_column = result1.pop('ship_name')  
result1.insert(2, 'ship_name', ship_name_column) 

In [39]:
#ds contient deux nouvelles colonnes avec l'information du nombre d'occurence des doublons de valeurs d'ais ainsi que l'info de si l'ais et un doublon ou non pour toutes les valeurs.
ds=pd.concat([result1,non_duplicates])
ds.sort_index(inplace=True)

In [119]:
"""
#liste des noms de bateaux
list_ship_name=duplicates['ship_name'].unique()
#
############Création des colonnes occurences et doublons dans le dataframe de valeurs d'ais doublons################################

list_Occ_ship=[]
for ship in list_ship_name :
    
    #Dataframe contenant toutes les lignes avec des valeurs d'ais non actualisées pour chaque bateau
    duplicates_sub=duplicates[duplicates['ship_name']==ship]
    #compter le nombre de fois qu'apparaît une même last_position_timepour un navire, (nombre d'occurences d'une valeur non actualisée d'ais)
    occurences=duplicates_sub['last_position_time'].value_counts()
    
    for i in range(len(occurences)):
        
        #Dataframe de toutes les occurences d'une valeur non actualisée d'ais
        Occ_ship_ij=duplicates_sub[duplicates_sub['last_position_time']==occurences.index[i]].copy()
         
        #Pour chaques last_position_time complêter les colonnes occurences et doublons
        Occ_ship_ij.loc[:,'occurences']=occurences.iloc[i]
        Occ_ship_ij.loc[:,'doublons']='réplicat'
        
        #la ligne de plus petit index est considéréé comme la dernière valeur d'ais actualisée
        Occ_ship_ij.loc[Occ_ship_ij.index[0],'doublons']='actualisée'
        
        #on ajoute à une liste tout les dataframes Occ_ship_ij
        list_Occ_ship.append(Occ_ship_ij)

#Dataframe de toutes les valeurs d'ais de tous les bateaux présentant des valeurs d'ais repetées
result = pd.concat(list_Occ_ship)


#réorganisation du Dataframe result afin de collre à la structure de non_duplicates 
result1 = result.drop('ship_name', axis=1)
result1.reset_index(level='ship_name', inplace=True)
ship_name_column = result1.pop('ship_name')  
result1.insert(2, 'ship_name', ship_name_column) 

#ds contient deux nouvelles colonnes avec l'information du nombre d'occurence des doublons de valeurs d'ais ainsi que l'info de si l'ais et un doublon ou non pour toutes les valeurs.
ds=pd.concat([result1,non_duplicates])
ds.sort_index(inplace=True)

"""

In [40]:
ds

,id,timestamp,ship_name,IMO,vessel_id,mmsi,last_position_time,position,speed,navigation_status,...,voyage_destination,voyage_draught,voyage_eta,accuracy,position_sensors,course,heading,rot,occurences,doublons
0,5997bb31-ec64-4c7b-8b41-1856ac09da85,2024-01-26 00:15:00,ANNELIES ILENA,9204556,1431,261084090,2024-01-23 11:11:01,0101000020E6100000184339D1AE521240ACD75F3C243B...,0.0,MOORED,...,IJMUIDEN,8.5,2023-12-12 02:00:00,LOW,TERRESTRIAL,324.4,212.0,0.000000,137,actualisée
1,0aaec083-c449-4698-9cdf-2b793b368ce8,2024-01-26 00:15:00,UK153 LUB SENIOR,9599731,1280,246749000,2024-01-25 18:49:58,0101000020E61000007F6993BBB664F93FA80018CFA05D...,0.0,ENGAGED_IN_FISHING,...,FISHING GROUND,5.0,2023-08-21 00:00:00,LOW,TERRESTRIAL,104.1,121.0,0.000000,57,actualisée
2,76ca3c41-7d70-4e7d-8e39-d0f21cfc6ff2,2024-01-26 00:15:00,SL-9 JOHANNA,7928835,1194,244938000,2024-01-25 17:13:17,0101000020E610000002DA1497FC62F93F225FD769035D...,0.0,NOT_DEFINED_DEFAULT,...,FISHING GROUNDS,4.0,NaT,NaN,SATELLITE,299.0,NaN,NaN,28,actualisée
4,944570d7-7279-4a2c-a00b-b151d413ed47,2024-01-26 00:15:00,HENDRIKA JACOBA,9877389,920,232031183,2024-01-25 05:08:33,0101000020E6100000A4880CAB78830040F47C9B3818D1...,0.4,ENGAGED_IN_FISHING,...,FISHING,4.6,2024-01-24 12:15:00,HIGH,TERRESTRIAL,53.0,150.0,0.000000,58,actualisée
5,63d11a78-88b3-422c-b95f-80030998a813,2024-01-26 00:15:00,F/V GEVRED,9741097,835,228066900,2024-01-26 00:04:32,0101000020E61000005BC104A808974C4074B7887B6ECE...,0.8,MOORED,...,PORT-LOUIS MU,8.0,2024-01-25 14:00:00,HIGH,SATELLITE,16.8,310.0,0.000000,4,actualisée
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8165,452503d2-6f8b-47fe-a9fc-2afa1758efef,2024-02-02 17:45:00,PARADANTA PRIMERO,8516043,621,224378000,2024-02-02 16:21:08,0101000020E610000089E4961C186B4EC085187D3F9419...,3.1,MOORED,...,CALADEROS,6.6,NaT,LOW,SATELLITE,225.0,232.0,0.000000,4,réplicat
8168,73c9220b-ecab-4c3e-97a7-3cc7f8bf2edc,2024-02-02 17:45:00,SCOMBRUS,9828936,784,226347000,2024-02-02 16:49:55,0101000020E61000008E40BCAE5F9015C0CDA4B870C13E...,9.1,UNDER_WAY_USING_ENGINE,...,FISHING GROUNDS,6.5,2024-02-01 11:11:00,LOW,SATELLITE,226.1,230.0,-16.115145,2,réplicat
8169,c8a49dcd-d69d-4a35-9a63-0c1caecfc067,2024-02-02 17:45:00,FRANCA MORTE,9297694,1507,263581000,2024-02-02 16:47:07,0101000020E61000000E032BC1418C46C0931D1B817825...,4.2,ENGAGED_IN_FISHING,...,NAFO,7.0,2024-01-09 08:00:00,HIGH,SATELLITE,86.7,NaN,NaN,2,réplicat
8170,bc8708af-b499-4e6b-80ca-a58095502802,2024-02-02 17:45:00,PRINS BERNHARD,8224406,795,227302000,2024-02-02 16:48:51,0101000020E61000007E52EDD3F1C82CC01BAEA3E4347B...,4.2,UNDER_WAY_USING_ENGINE,...,FISHING GROUNDS,6.2,2024-01-31 18:30:00,LOW,SATELLITE,46.7,41.0,0.000000,2,réplicat
